# Multiple Linear Regression

So far we were just looking on simple linear regression problems. But within a linear regression model there could also be used more than one independent variable to describe the dependent varialbe. 

## Learning Objectives

At the end of this notebook you should be able to
- Apply multiple linear regression in python
- Interprete multiple linear regression models
- Explain why it is better to use **adjusted $R^2$** than $R^2$ to compare multiple linear regression models

## The Multiple Regression Model

Multiple linear regression is very similar to simple linear regression except that the dependent variable $y$ is described by $k$ independent variables $x_1, \dots, x_k$  

$$ y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_k x_k + \varepsilon $$  

**Our predicted model**  

$$\hat{y} = b_0 + b_1 x_1 + b_2 x_2 + \dots + b_k x_k$$

* The interpretation of the intercept is the same  
* Slope $b_i$ is the change in $y$ given a unit change in $x_i$ while **holding all other variables constant**  




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

%matplotlib inline

In [ ]:
#using the same car dataset as before
cars = pd.read_csv("data/cars_multivariate.csv",
                  na_values='?')
cars.head(n=2)

In [ ]:
cars.plot('horsepower', 'mpg', kind='scatter');

## Define model

As before, we will first use a simple linear regression model and see how well it fits our data.

In [ ]:
import statsmodels.api as sm

In [ ]:
X = cars.horsepower
X = sm.add_constant(X)
y = cars.mpg

sm.OLS(y,X).fit().summary()

This same model can be fitted using **another syntax** to define model:


In [ ]:
# "formula notation" (similar to R)
import statsmodels.formula.api as smf
# Define the model, fit the model, and generate the summary
smf.ols(formula='mpg ~ horsepower', data=cars).fit().summary()

Did you notice, hat with this syntax we didn't need to add a "constant" to calculate the intercept? 

## Multiple Regression  

Instead of using `horsepower` as the only independent variable to predict `mpg`, we might want to include other independent variables in the model.  

Let's try to add `weight` to the model.

In [ ]:
sns.lmplot('weight', 'mpg',data=cars);

In [ ]:
X2 = cars[['horsepower', 'weight']]
X2 = sm.add_constant(X2)
y2 = cars.mpg

X2.head(n=2)

In [ ]:
sm.OLS(y2, X2).fit().summary()

Does this Model describe the variance of `mpg` better than the simple linear regression?

Our model explains about 70% of the variation in `mpg`.

Our multiple regression model is give by  

$$ \hat{mpg} = 45.6402 - 0.0473 \times horsepower - 0.0058 \times weight $$

What is the predicted `mpg` for a car with 200 `horsepower` and a `weight` of 3500?

$$ \hat{mpg} = 45.6402 - 0.0473 (200) - 0.0058 (3500) = 15.88 $$  


We would expected the `mpg` of the car to be 15.88.
This can of course also be calculated with our model. We only need to call the `.predict()`function and name the values of X.

In [ ]:
test_car = [1,200,3500]
sm.OLS(y2, X2).fit().predict(test_car)

## Model Interpretation

**Intercept** 

The mean value for $y$ when all $x=0$


**slope**:

For each unit increase in $x$, the expected increase/decrease in $y$ 

In the case of multiple linear regression, we need to add "holding all other explanatory variables constant", since there are more than one exaplanatory variable in the model.



In [ ]:
# The coefficients
sm.OLS(y2, X2).fit().params

We would expect `mpg` to decrease by 0.047 as `horsepower` goes up by 1,  **holding `weight` constant**.

We would expect `mpg` to decrease by 0.006 as `weight` increases by 1, **holding `horsepower` constant**, 

We can also use the other model syntax we have seen before with multiple linear regression:

In [ ]:
# mpg explained by horsepower and weight
smf.ols(formula='mpg ~ horsepower + weight', data=cars).fit().summary()

Explore Multiple Regression
-------

A really graphical way of explaining Ordinary Least Squeares can be found here:
[OLS Explained Visually](http://setosa.io/ev/ordinary-least-squares-regression/)

Unfortunately, multiple linear regression is not so easy to display, but the use of only two independent variables predicting a dependent variable can still be displayed graphically with a 3D plot.

How to built this 3D-plot is explained in this link: [Datarobot Multiple Regression Appendix, how to create a three dimensional hyperplane plot](https://www.datarobot.com/blog/multiple-regression-using-statsmodels/#appendix).


**Example**  

Now let's try to predict `mpg` using `displacement`, `horsepower`, `weight` and `acceleration`. 

In [ ]:
cars[['displacement', 'horsepower', 'weight', 'acceleration']].corr()

You see that there are quite some correlations between these variables! These correlations can also be seen in the scatter plots:

In [ ]:
X3 = cars[['displacement', 'horsepower', 'weight', 'acceleration']]
sns.pairplot(X3);


In [ ]:
X3 = sm.add_constant(X3)
y3 = cars.mpg

model3 = sm.OLS(y3, X3)
results3 = model3.fit()
results3.summary()

You should always question your model. Here are some questions you should be able to anser:

How good is the model fit?

What is our regression equation?



<br>

<details><summary>
Click here for the solution
</summary>
(copy the lines below to a markdown cell)
$$ \hat{mpg} = 45.2511 - 0.0060 \times displacement - 0.0436 \times horsepower - 0.0053 \times weight - 0.0231 \times acceleration $$
</details>


How do we interpret the model?

Is it a good idea to include `displacement` in the model?

## $R^2$  


 $R^2$ leads to an overwhelming temptation to put lots of $X$’s in.  

This is a not great idea 💡. We want to summarize and predict - __and__ we want to do it in the simplest possible way. 

In [ ]:
rs = smf.ols(formula='mpg ~ horsepower', data=cars).fit().rsquared
f'{rs:.3}'

In [ ]:
rs = smf.ols(formula='mpg ~ horsepower + weight', data=cars).fit().rsquared
f'{rs:.3}'

In [ ]:
rs = smf.ols(formula='mpg ~ displacement + horsepower + weight + acceleration', data=cars).fit().rsquared
f'{rs:.3}'

## Adjusted $R^2$  

The adjusted $R^2$ is designed to build in an automatic penalty for adding an $X$.  

$$R_a^2 = 1 - \frac{\frac{1}{n - k - 1} SSE}{\frac{1}{n - 1} SST} $$  

We use adjusted $R^2$ when we compare models with different numbers of independent variables.

$R^2$ penalizes model complexity (to control for overfitting), but it generally [under-penalizes complexity](http://scott.fortmann-roe.com/docs/MeasuringError.html).

According to the adjusted $R^2$, which is the best model we have fitted so far? Calculate adjusted $R^2$ for the former three models!

Summary
-----

- We extend the linear regression model to include many explanatory variables (Xs).
- All explanatory variables should be independent of each other.
- $R^2$ allows use to measure how good a model fits the data.
- Adjusted $R^2$ penalizes adding non-useful explanatory variables. And is used to compare models with different numbers of independent variables.

----
Bonus Material
----

### The Overall F Test  

Instead of testing if one independent variable is useful for predicting the dependent variable, we might want to test if the model as a whole is useful for predicting the dependent variable.  

For that, we use the F test.  

* It tests the null hypothesis:  

$$ H_0: \beta_1 = \beta_2 = \dots = \beta_k = 0$$  

* Under the null hypothesis, there are no $X$ variables in the model.  

* The alternative hypothesis:  

$$ H_a: \text{ at least one } \beta_i \neq 0 $$  

**Test statistic**  

$$f = \frac{\frac{SSR}{k}}{\frac{SSE}{(n - k - 1)}}$$ 

The larger the value of $f$, the stronger the evidence to reject the null.  

The test statistic follows an F distribution, and we find the p-value by comparing the test statistic to an F distribution with degress of freedom $k$ and $n - k - 1$.   

We will simply read the p-value from the regression output. (Score!)

In [ ]:

results3.summary()

$$\text{F-statistic} = 233.4$$
$$\text{p-value} = 9.63 \times 10^{-102} $$   

We have extremely strong evidence to conclude that the at least one of the variables in the model is useful for predicting `mpg`.

Is Model 1 (with one variable `horsepower`) useful for predicting `mpg`?

In [ ]:
# Add the model here and answer the questions by looking at the Regression Results
results.summary()

Is Model 2 (with two variables `horsepower` and `weight`) useful for predicting `mpg`?

In [ ]:
# Add the model here and answer the questions by looking at the Regression Results
results2.summary()

In [ ]:
X4 = cars[['displacement', 'horsepower', 'acceleration']]

X4 = sm.add_constant(X4)
y3 = cars.mpg

model4 = sm.OLS(y3, X4)
results4 = model4.fit()
results4.summary()